In [1]:
print 'load libraries...'

load libraries...


In [2]:
# Aim: Obtains the Retail Loan Customers' "Account" and "Customer/Biodata" Info 

In [3]:
# Add path in order to load utility functions
import sys
sys.path.append(r"N:\18. Hot Spot\Custom Modules\Utils")
sys.path.append(r"N:\18. Hot Spot\Custom Modules\ETLMonthly")

In [4]:
# setup environment and import libraries
import os
import re
import datetime
import time
import getpass
import pandas as pd
import numpy as np
from datetime import datetime, date
from datetime import timedelta
import calendar as cd
from collections import OrderedDict
path=r'C:\ProgramData\Anaconda2\instantclient_12_2'
os.environ['PATH']=path
import cx_Oracle
from sqlalchemy import create_engine
import file_utils
import db_utils
import datetime_utils
import config
#set working directory
os.chdir(config.working_dir)

In [5]:
# pd.set_option('display.max_columns', 500)

In [6]:
# db credentials authentication

In [7]:
print 'connect to database...'
conn_string = db_utils.db_connect(config.db_username,config.db_pwd)

connect to database...
Testing Connection...
Connection Succesful!


In [8]:
mtch = re.match('.*@', conn_string)
un, pwd = mtch.group().rstrip('@').split('/')

#### Get Retail Customer Info

In [9]:
con = cx_Oracle.connect(conn_string)
retail_loan_cust = pd.read_sql_query('select bra_code, cus_num from cs_retail_loan_cust', con)
con.close()

len(retail_loan_cust)

154724

#### Get Customers BioData

In [10]:
print 'get loan customers customer info...'

con = cx_Oracle.connect(conn_string)
df_retail_loan_cust_info = pd.read_sql_query(r'''
select a.bra_code, a.cus_num, a.date_open cus_date_open, a.cus_gender, b.edu_lvl,
a.cus_marital_status, b.prof_code, c.city, c.state, a.bir_date, a.cust_grade_seg,
(case when a.cust_grade_seg in (1,4) then 1 else 0 end) is_platinum_plus_hni,
(case when a.cus_class = 7 then 1 else 0 end) is_pep
from cs_retail_loan_cust l
left join stg.src_customer_extd@exadata_new a
on l.bra_code=a.bra_code and l.cus_num=a.cus_num
left join stg.src_cust_pro b
on l.bra_code=b.bra_code and l.cus_num=b.cus_num
left join (select tab_ent, des_eng as city, con_val_2 as state from stg.src_text_tab where tab_id=190) c
on a.city_loc_code=c.tab_ent
''', con)
con.close()

get loan customers customer info...


In [11]:
print len(df_retail_loan_cust_info)

154724


In [12]:
df_retail_loan_cust_info.to_csv('cs_retail_loan_cust_info.csv', index=False)

#### Does customer has a business account

In [13]:
# is_director or has_business_account_with_gtb

con = cx_Oracle.connect(conn_string)
df_retail_loan_cust_has_biz_acc = pd.read_sql_query(r'''
select distinct a.bra_code, a.cus_num, nvl2(c.bvn,1,0) has_biz_acc
--c.bra_code biz_bra_code, c.cus_num biz_cus_num,
--c.bvn, c.cus_name as business_name
from cs_retail_loan_cust a
left join
(select bra_code, cus_num, cus_name, bvn
from stg.src_customer_extd@exadata_new where cust_grade_seg in (1,2,3,4,5)) b
on a.bra_code=b.bra_code and a.cus_num=b.cus_num
left join
(select bra_code, cus_num, cus_name, bvn
from stg.src_customer_extd@exadata_new where cust_grade_seg>5
and cus_name not like 'EST%') c
on b.bvn=c.bvn''', con)
con.close()

In [14]:
print len(df_retail_loan_cust_has_biz_acc)

154724


In [15]:
df_retail_loan_cust_has_biz_acc.to_csv('cs_retail_loan_cust_has_biz_acc_info.csv', index=False)

#### Create a table on database for retail loan customers' accounts - a month before the loan month

In [16]:
# The aaccount info picked per customer is as at dynamic periods.
# Customer account info was subject to the previous month to the date they borrowed the last loan
# i.e. a customer whose latest loan was on 22feb2016, will have account info as at 28feb2016 (month end)

In [17]:
print 'get accounts of retail loan customers...'

cs_retail_loan_cust_accs = pd.DataFrame()

for d_range in datetime_utils.first_last_day_of_months(datetime(2018,1,1), datetime.strptime(config.ref_date, '%d%b%Y')):
    cur_reference_date = datetime.strftime(d_range[1],'%d%b%Y')
    print cur_reference_date
    
    con = cx_Oracle.connect(conn_string)
    cs_retail_loan_cust_accs_chunk = pd.read_sql_query('''
    select a.date_open loan_date_open, add_months(last_day(a.date_open),-1) prior_month_to_loan,
    b.reference_date, a.is_bad, b.bra_code, b.cus_num, b.cur_code, b.led_code,
    b.sub_acct_code, b.date_open, 
    b.sta_code, b.date_sta_cha, b.lper_ave_bal, b.crnt_bal, b.pre_day_crnt_bal, b.cre_tra_num, b.deb_tra_num, 
    b.las_tra_date, eco_sec, type_of_dep, b.num_of_che_book
    from cs_retail_loan_cust a
    inner join stg.d_account_snapshot b
    on a.bra_code=b.bra_code and a.cus_num=b.cus_num
    and b.reference_date='{}'
    '''.format(cur_reference_date),
    con)
    con.close()
    
    cs_retail_loan_cust_accs_chunk = cs_retail_loan_cust_accs_chunk[cs_retail_loan_cust_accs_chunk.REFERENCE_DATE== \
                                                                   cs_retail_loan_cust_accs_chunk.PRIOR_MONTH_TO_LOAN]
    
    cs_retail_loan_cust_accs = pd.concat([cs_retail_loan_cust_accs,cs_retail_loan_cust_accs_chunk])

get accounts of retail loan customers...
31Jan2018
28Feb2018
31Mar2018
30Apr2018
31May2018
30Jun2018
31Jul2018
31Aug2018
30Sep2018
31Oct2018
30Nov2018
31Dec2018
31Jan2019
28Feb2019
31Mar2019
30Apr2019
31May2019
30Jun2019
31Jul2019
31Aug2019
30Sep2019
31Oct2019
30Nov2019
31Dec2019
31Jan2020
29Feb2020


In [25]:
cs_retail_loan_cust_accs.to_csv('cs_retail_loan_cust_accs.csv', index=False)

In [26]:
loan_cust_accs_numeric_cols = ['BRA_CODE','CUS_NUM','CUR_CODE','LED_CODE','SUB_ACCT_CODE','STA_CODE','CRE_TRA_NUM',
                                 'DEB_TRA_NUM','TYPE_OF_DEP','NUM_OF_CHE_BOOK']

loan_cust_accs_date_cols = ['LOAN_DATE_OPEN','PRIOR_MONTH_TO_LOAN','REFERENCE_DATE','DATE_OPEN','LAS_TRA_DATE','DATE_STA_CHA']

cs_retail_loan_cust_accs[loan_cust_accs_numeric_cols] =  \
        cs_retail_loan_cust_accs[loan_cust_accs_numeric_cols].apply(pd.to_numeric)
cs_retail_loan_cust_accs[loan_cust_accs_date_cols] = \
        cs_retail_loan_cust_accs[loan_cust_accs_date_cols].apply(pd.to_datetime, errors='coerce')


In [27]:
conn = create_engine('oracle+cx_oracle://{}:{}@dwdrscan.gtbank.com:1521/?service_name=odidb'.format(un, pwd))    
cs_retail_loan_cust_accs.to_sql('cs_retail_loan_cust_accs', conn, index=False, if_exists='replace', chunksize=10000,
                          dtype=db_utils.sqlcol(cs_retail_loan_cust_accs))

#### Get Account Info for the Retail Loan Customers

In [10]:
print 'get account info (derived info) of retail loan customers...'

get account info (derived info) of retail loan customers...


In [11]:
# Extract the retail loan ledgers that will be considered during the analysis
# retail_loan_ledgers = pd.read_excel(r"..\Docs\Retail loan ledgers.xlsx", sheetname="SQL Results")['LED_CODE']
retail_loan_ledgers = config.retail_loan_ledgers
retail_loan_ledgers = ','.join([str(l) for l in retail_loan_ledgers])

In [12]:
dep_ledgers = '''1,2,4,6,8,9,13,16,23,24,26,51,55,57,59,60,61,65,67,
68,69,82,5001,5012,5020,5022,5023,5024,5025,5027,5032,5033,5115,5116'''

In [13]:
call_time_tbills_ledgers = '55,57,5022,5023,3005,3006'

In [14]:
ijebu_ledgers = '55,57,5022,5023,3005,3006,5033,24'

In [15]:
print retail_loan_ledgers, '\n\n', dep_ledgers, '\n\n', call_time_tbills_ledgers

1118,1121 

1,2,4,6,8,9,13,16,23,24,26,51,55,57,59,60,61,65,67,
68,69,82,5001,5012,5020,5022,5023,5024,5025,5027,5032,5033,5115,5116 

55,57,5022,5023,3005,3006


In [16]:
con = cx_Oracle.connect(conn_string)
df_retail_loan_cust_acc_info = pd.read_sql_query(
'''
select a.bra_code, a.cus_num,
count(distinct case when led_code in ({retail_loan_ledgers}) then led_code end) nbr_distinct_loan,
count(case when led_code in ({retail_loan_ledgers}) then led_code end) nbr_loans,

abs(nvl(sum(case when led_code in ({retail_loan_ledgers}) and pre_day_crnt_bal<0 then pre_day_crnt_bal end),0)) outstanding_debt,
abs(nvl(sum(case when led_code in ({dep_ledgers}) and pre_day_crnt_bal<0 then pre_day_crnt_bal end),0)) due_debt,

count(distinct case when led_code in ({retail_loan_ledgers}) and sta_code=1 then led_code end) nbr_distinct_active_loan,
count(case when led_code in ({retail_loan_ledgers}) and sta_code=1 then led_code end) nbr_active_loans,

count(distinct case when led_code in ({dep_ledgers}) then led_code end) nbr_distinct_dep,
count(case when led_code in ({dep_ledgers}) then led_code end) nbr_deps,

count(distinct case when led_code in ({call_time_tbills_ledgers}) then led_code end) nbr_distinct_call_time_tbills,
count(case when led_code in ({call_time_tbills_ledgers}) then led_code end) nbr_call_time_tbills,

count(distinct case when led_code in ({ijebu_ledgers}) then led_code end) nbr_distinct_ijebu_ledgers,
count(case when led_code in ({ijebu_ledgers}) then led_code end) nbr_ijebu_ledgers,

nvl(avg(case when date_sta_cha>'01jan1900' and sta_code=1 and led_code in ({dep_ledgers}) then 1 end),0) has_reactivated,
nvl(avg(case when date_sta_cha>=trunc(add_months(a.date_open,-12),'month')
    and sta_code=1 and led_code in ({dep_ledgers}) then 1 end),0) has_reactivated_12_mths,

count(distinct case when cur_code>1 then 1 end) nbr_distinct_dom_acc,
count(case when cur_code>1 then 1 end) nbr_dom_accs,

nvl(sum(case when led_code in ({dep_ledgers}) and lper_ave_bal>0 then lper_ave_bal end),0) avg_bal_all_acc,
nvl(sum(case when led_code in ({dep_ledgers}) and pre_day_crnt_bal>0 then pre_day_crnt_bal end),0) crnt_bal_all_acc

from (select bra_code, cus_num, date_open from cs_retail_loan_cust) a
left join cs_retail_loan_cust_accs b
on a.bra_code=b.bra_code and a.cus_num=b.cus_num
group by a.bra_code, a.cus_num
'''.format(
    retail_loan_ledgers=retail_loan_ledgers,
    dep_ledgers=dep_ledgers,
    call_time_tbills_ledgers=call_time_tbills_ledgers,
    ijebu_ledgers=ijebu_ledgers), con)
con.close()

print len(df_retail_loan_cust_acc_info)

154724


In [17]:
df_retail_loan_cust_acc_info.to_csv('cs_retail_loan_cust_acc_info.csv', index=False)